In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cv2
import random
import tensorflow as tf
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras import Sequential

In [2]:
classes = { 0:'Speed limit (20km/h)',
            1:'Speed limit (30km/h)', 
            2:'Speed limit (50km/h)', 
            3:'Speed limit (60km/h)', 
            4:'Speed limit (70km/h)', 
            5:'Speed limit (80km/h)', 
            6:'End of speed limit (80km/h)', 
            7:'Speed limit (100km/h)', 
            8:'Speed limit (120km/h)', 
            9:'No passing', 
            10:'No passing veh over 3.5 tons', 
            11:'Right-of-way at intersection', 
            12:'Priority road', 
            13:'Yield', 
            14:'Stop', 
            15:'No vehicles', 
            16:'Veh > 3.5 tons prohibited', 
            17:'No entry', 
            18:'General caution', 
            19:'Dangerous curve left', 
            20:'Dangerous curve right', 
            21:'Double curve', 
            22:'Bumpy road', 
            23:'Slippery road', 
            24:'Road narrows on the right', 
            25:'Road work', 
            26:'Traffic signals', 
            27:'Pedestrians', 
            28:'Children crossing', 
            29:'Bicycles crossing', 
            30:'Beware of ice/snow',
            31:'Wild animals crossing', 
            32:'End speed + passing limits', 
            33:'Turn right ahead', 
            34:'Turn left ahead', 
            35:'Ahead only', 
            36:'Go straight or right', 
            37:'Go straight or left', 
            38:'Keep right', 
            39:'Keep left', 
            40:'Roundabout mandatory', 
            41:'End of no passing', 
            42:'End no passing veh > 3.5 tons' }

In [3]:
train_df = pd.read_csv('Datensatz/Train.csv')
test_df = pd.read_csv('Datensatz/Test.csv')

In [4]:
train_df.head()

,Width,Height,Roi.X1,Roi.Y1,Roi.X2,Roi.Y2,ClassId,Path
0,27,26,5,5,22,20,20,Train/20/00020_00000_00000.png
1,28,27,5,6,23,22,20,Train/20/00020_00000_00001.png
2,29,26,6,5,24,21,20,Train/20/00020_00000_00002.png
3,28,27,5,6,23,22,20,Train/20/00020_00000_00003.png
4,28,26,5,5,23,21,20,Train/20/00020_00000_00004.png


In [5]:
test_df.head()

,Width,Height,Roi.X1,Roi.Y1,Roi.X2,Roi.Y2,ClassId,Path
0,53,54,6,5,48,49,16,Test/00000.png
1,42,45,5,5,36,40,1,Test/00001.png
2,48,52,6,6,43,47,38,Test/00002.png
3,27,29,5,5,22,24,33,Test/00003.png
4,60,57,5,5,55,52,11,Test/00004.png


In [6]:
train_df['Path'] = train_df['Path'].apply(lambda x: 'Datensatz/' + x)
test_df['Path'] = test_df['Path'].apply(lambda x: 'Datensatz/' + x)

In [7]:
random.choice(train_df['Path'].tolist())

'Datensatz/Train/1/00001_00026_00019.png'

In [8]:
random_sample = train_df.sample(16)
plt.figure(figsize=(20,20))
for i in range(1,17):
    plt.subplot(4,4,i)
    img = plt.imread(random_sample['Path'].values.tolist()[i-1])
    plt.imshow(img)
    plt.grid(False)
    plt.xlabel(classes[random_sample['ClassId'].values.tolist()[i-1]], fontsize = 20)
    plt.ylabel(img.shape[0], fontsize = 20)

In [ ]:
train_data = train_df.sample(frac=0.8,random_state=42)
val_data = train_df.drop(train_data.index)

NUM_CLASSES = len(classes)
IMG_SIZE = 32

def preprocess_image(image):
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    image = image/255
    return image

def load_data(data):
    X = []
    y = []
    for path, label in zip(data['Path'].values.tolist(), data['ClassId'].values.tolist()):
        image = cv2.imread(path)
        image = preprocess_image(image)
        X.append(image)
        y.append(label)
    X = np.array(X)
    y = np.array(y)
    return X, y

X_train, y_train = load_data(train_data)
X_val, y_val = load_data(val_data)
X_test, y_test = load_data(test_df)

In [ ]:
print(f'The shape of X_train : {X_train.shape}')
print(f'The shape of X_test : {y_train.shape}')

The shape of X_train : (31367, 32, 32, 3)
The shape of X_test : (31367,)
